# Scrapper for guitar selling posts

This script scraps both *The Gear Page* forum and the *Gear4Sale* subreddit for posts selling guitars.
For now the below characteristics are scrapped:
- brand
- price
- vintage
- color
- wood

To do:
- scrap model
- create database
- compute average/median price
- scrap prices from other places ?
- define if a model is "a good buy opportunity"

It then saves the posts in a csv file.

## FYI Persistence

scrappedGuitars:([] id:`symbol$();brand:`symbol$();model:`symbol$();price:`float$();vintage:`int$();color:`symbol$();wood:`symbol$();website:`symbol$();url:();title:();content:())
recommendedGuitars:([] id:`symbol$())

## Set up

Be sure to have the below path created, or to modify it for the data to be saved where you want

In [ ]:
path="C:/dev/niche/guitar/data/"

Be sure to be able to import the below packages

In [214]:
import bs4
import datetime
import os
import pandas as pd
from qpython import qconnection
import requests
from uuid import uuid4

## Functions

In [394]:
def get_url(section):
    a_s=section.findAll("a")
    for a in a_s:
        if a.has_attr('href'):
            # the first link is the right one
            return a["href"]
        #TODO check link is valid

def validate_string(s):
    """
    Returns True if:
        - s is a string
    &
        - s is not an empty string
    """
    is_s=False
    if type(s)==str:
        if s!="":
            is_s =True
    return is_s

def find_vintage(content):
    result = ""
    content = content.lower()
    # from 1985
    if "from " in content:
        split_on_from =  content.split("from ")
        if len(split_on_from)==2:
            #"guitar from 1985"
            year = split_on_from[1][:4]
            # still check that we are returning a number
            if year.isdigit():
                if check_vintage(year):
                    result = year
        else:
            # "guitar from the best from 1985"
            for i in range(len(split_on_from)):
                year = split_on_from[i][:4]
                # still check that we are returning a number
                if year.isdigit():
                    if check_vintage(year):
                        result = year
    else: 
        potential_years = []
        for i in range(len(content)):
            if len(content[i:])>4:
                four_char = content[i:i+4]
                if four_char.isdigit():
                    if check_vintage(four_char):
                        potential_years.append(four_char)
            if len(potential_years)==1:
                # "guitar blabla 1985"
                result=potential_years[0]
            elif len(potential_years)>1:
                # "guitar made in 1985, bought in 2001"
                result = min(potential_years)
    return result
        # TODO check for "made in", "manufactured in" etc

def find_longest_string_of_digits(s):
    """
    from a string, assuming the first character is a digit, will return the longest number
    eg:
    - "1234 frefr" -> "1234"
    - "j 12345 p" -> ""
    
    :s: string
    """
    i=0
    n = len(s)
    if n>0:
        while (s[i].isdigit() or (s[i]==".") or (s[i]==",")) and (i<n-1):
            i+=1
            if (i==n):
                break
        return s[:i]
    else:
        return ""

def find_price(content):
    result = ""
    content = content.lower()
    if "price is " in content:
        split_on_price_is = content.split("price is ")
        if len(split_on_price_is)==2:
            # price is 300,45
            # find
            p = find_longest_string_of_digits(split_on_price_is[1][find_ind_of_next_digit_in_string(split_on_price_is[1]):])
            if check_price(p):
                result = p
        else:
            ps = []
            # price is 45.00. price is negotiable:
            for i in range(len(split_on_price_is)):
                print(split_on_price_is[i])
                longest_digit = find_longest_string_of_digits(split_on_price_is[i])
                if longest_digit!="":
                    if check_price(longest_digit):
                        ps.append(longest_digit)
            if len(ps)==1:
                result = ps[0]
            elif len(ps)>1:
                result = ps[0]
    for sign in ["usd", "$", "price"]:
        if sign in content:
            split_on_dollar_sign = content.split(sign)
            p = find_longest_string_of_digits(split_on_dollar_sign[1][find_ind_of_next_digit_in_string(split_on_dollar_sign[1]):])
            if p.isdigit():
                if check_price(p):
                    return p
    return result

def check_price(p,lb=100,ub=5000):
    if type(p)==str:
        p = float(p.replace(",",""))
    return p>lb  and p <ub

def check_vintage(v):
    if type(v)==str:
        v=int(v)
    return v>1900 and v<2021

def find_ind_of_next_digit_in_string(s):
    """
    Returns the index of the next character which is a digit
    
    eg:
    - "Ah 1" -> 3
    - "2" -> 0
    - "ABC" -> None
    
    """
    for i in range(len(s)):
        if s[i].isdigit():
            return i
        
def get_brand_and_model(brand_models,s):
    """
    s: string, content to look into
    brand_models:dict, eg:
        {"brand1":["model1","model2"],
        "brand2":["model1","model2"],
        "brand3":["model1","model2"],}
        
    return: {"brand":["brand1","brand2"], "model":["model1","model2"]}
    """
    result={"brand":[],"model":[]}
    for brand in brand_models.keys():
        if brand in s:
            result["brand"].append(brand)
    if result["brand"]!=[]:
        for brand in result["brand"]:
            for model in brand_models[brand]:
                if model in s:
                    result["model"].append(model)
    return result

def get_brand_model_from_title_content(brand_models, title, content):
    brand=""
    model=""
    brand_model = get_brand_and_model(brand_models, title)
    if len(brand_model["brand"])==1:
        brand = brand_model["brand"][0]
    if len(brand_model["model"])==1:
        model = brand_model["model"][0]
    
    # if I did not manage to get brand from title, then get it from content
    if brand=="":
        brand_model = get_brand_and_model(brand_models, content)
        if len(brand_model["brand"])==1:
            brand = brand_model["brand"][0]
        # and get model as well
        if len(brand_model["model"])==1:
            model = brand_model["model"][0]    
    return brand, model

def get_brand(content):
    brands = pd.read_csv(path+"guitar_brands.csv")["Brand"].tolist()
    result = ""
    content = content.lower()
    for brand in brands:
        if brand in content:
            return brand
    return result

def get_model(content,brand):
    # TODO: for each brand get a list of the models and save it in a csv: brand_models.csv
    result = ""
    models = []#pd.read_csv(path+brand+"_models.csv")["Model"].tolist()
    content = content.lower()
    for model in models:
        if model in content:
            return model
    return result
    
def find_color(s):
    for c in ["blue", "red", "yellow", "pink", "green", "purple", "white", "grey", "black", "beige", "orange"]:
        if c in s:
            return c
        
def get_wood(s):
    # TODO: improve this function by getting the wood for the different parts:
    #eg:
    #   Top: Basswood
    #   Body: Basswood
    #   Neck: Mahogany
    #   Fingerboard: Rosewood
    #   Nut: Bone
    
    woods = ["alder","basswod","magahony","swamp ash", "walnut","koa","mapple","rosewood","ebony","wenge"]
    result = ""
    s=s.lower()
    for wood in woods:
        if wood in s:
            return wood
    if "neck wood" in s:
        split_on_neck_wood = s.split("neck wood")
    return result

def find_relevant_information(content,title,url,post_date):
    # ["brand","model","price","vintage","color","wood","post_date", "post_title","post_content","post_url"]
    new_row = []
    
    # find vintage
    vintage = find_vintage(title)
    if vintage =="":
        vintage = find_vintage(content)
    # find price
    price = find_price(content)

    # If we have price, find brand, model, wood, color
    if price!="":

        # find brand
        brand = get_brand(content)
        if brand=="":
            brand = get_brand(title)
        # find model
        model = get_model(content, brand)
        if model=="":
            model = get_model(content, brand)
        # find color
        color = find_color(content)
        # find wood
        wood = get_wood(content)

        # create a new pd.Df row and insert it if not only there
        new_row=[brand,model,price, vintage,color,wood,post_date,title,content,url,]
        
    return new_row

def scrap_brands_from_wikipedia():
    """
    Scrapp all guitar brands from wikipedia and saves it into a file
    """
    brands = []
    wikipedia_url = "https://en.wikipedia.org/wiki/List_of_guitar_manufacturers"
    wikipedia_rep = requests.get(wikipedia_url)
    wikipedia_soup = bs4.BeautifulSoup(wikipedia_rep.text, 'html.parser')
    lis = wikipedia_soup.findAll("div",{"class": "div-col", "style":"column-width: 22em;"})[0].findAll("li")
    for li in lis:
        a_s = li.findAll("a")
        if len(a_s)>0:
            if a_s[0].has_attr('title'):
                title = str(a_s[0]["title"]).lower()
                # machin (bidule) -> machin
                title = title.split(" (")[0]
                # machin guitars -> machin
                title = title.split(" guitars")[0]
                title = title.split(" guitar")[0]
                # machin music -> machin
                title = title.split(" music")[0]
                brands.append(title)
    pd.DataFrame(brands, columns=["Brand"]).to_csv(path+"guitar_brands.csv", index=False)
    return brands

def kdb_format_year(year):
    year = str(year)
    y = ""
    for char in year:
        if char.isdigit():
            y+=char
    year=y
    if year=="None" or year =="":
        return "0Ni"
    else:
        return year+"i"
    
def insert_scrappedGuitars_into_kdb(df):
    for index, row in df.iterrows():
        kdb_row  = str(row["post_date"])+";"+\
            "`"+str(uuid4()).replace("-","")+";"+\
            "`$\""+str(row["brand"])+"\""+";"+\
            "`$\""+str(row["model"])+"\""+";"+\
            str(float(row["price"].replace(",","")))+";"+\
            kdb_format_year(str(row["vintage"]))+";"+\
            "`$\""+str(row["color"])+"\""+";"+\
            "`$\""+str(row["wood"])+"\""+";"+\
            "`$\""+str(row["website"])+"\""+";"+\
            "\""+str(row["post_url"])+"\""+";"+\
            "\""+str(row["post_title"]).replace("\"","").replace(u"\u2019"," ")+"\""

        with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
            #scrappedGuitars:([] date:`date$();id:`symbol$();brand:`symbol$();model:`symbol$();price:`float$();vintage:`int$();color:`symbol$();wood:`symbol$();website:`symbol$();url:();title:())
            try:
                X=q('upsert[`scrappedGuitarsToday;('+ kdb_row +')]')
            except Exception as error:
                print(error)
                print('('+ kdb_row +')')
                print("___")
                break
                
def init_kdb_instance():
    init="""
    scrappedGuitarsToday:([] date:`date$();id:`symbol$();brand:`symbol$();model:`symbol$();price:`float$();vintage:`int$();color:`symbol$();wood:`symbol$();website:`symbol$();url:();title:());
    recommendedGuitarsToday:([] date:`date$();id:`symbol$());
    
    hdbDirectory : `$"c:/dev/niche/kdb";
    system "l ",string hdbDirectory;
    rollToHdb:{[hdbDirectory;data;targetDate]
        dataTargetDate:select from data where date=targetDate;
        if[0<count dataTargetDate;
            $[2<count cols dataTargetDate;
                [`scrappedGuitars set `brand`model xasc delete date from data;
                .Q.dpft[hsym hdbDirectory;targetDate;`brand;`scrappedGuitars]];
                [`recommendedGuitars set `id xasc delete date from data;
                .Q.dpft[hsym hdbDirectory;targetDate;`id;`recommendedGuitars]]
                ];
            ];
        };

    """

    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
            try:
                X=q(init)
            except Exception as error:
                print(error)

def roll_to_HDB_all_dates():
    roll = "rollToHdb[hdbDirectory;scrappedGuitarsToday;] each exec distinct date from scrappedGuitarsToday;"+\
           "rollToHdb[hdbDirectory;recommendedGuitarsToday;] each exec distinct date from recommendedGuitarsToday;"
    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
        try:
            X=q(roll)
        except Exception as error:
            print(error)
            
def roll_to_HDB_today_only():
    roll = "rollToHdb[hdbDirectory;scrappedGuitarsToday;.z.D]" +\
           "rollToHdb[hdbDirectory;recommendedGuitarsToday;.z.D]"
    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
        try:
            X=q(roll)
        except Exception as error:
            print(error)

## REDDIT

In [337]:
df_reddit=pd.DataFrame(columns=["brand","model","price","vintage","color","wood","post_date","post_title","post_content","post_url"])
k=0
n_pages = 5

print("Starting to scrap The Gear Page, processing the first "+str(n_pages)+" pages.")
after = ""
for page_number in [i+1 for i in range(n_pages)]:
    url = 'https://www.reddit.com/r/Gear4Sale/.json'
    
    if after!="":
        url += '?&after='+after
    
    r = requests.get(
        url,
        headers={'user-agent': 'Mozilla/5.0'})
    
    data = r.json()['data']
    all_posts =data['children']
    after = data["after"]

    for post in all_posts:
        title = post["data"]["title"]
        if any(ext in title for ext in ["sell", "wts","Sell","WTS"]):
            title = post["data"]["title"]
            content = post["data"]["selftext"]
            url = post["data"]["url"]
            post_date = datetime.datetime.fromtimestamp(float(post["data"]["created"])).strftime("%Y.%m.%d")
            # create a new pd.Df row and insert it if not only there
            new_row = find_relevant_information(content,title,url,post_date)
            if new_row!=[] and not ((df_reddit['post_title'] == title) & (df_reddit['post_content'] == content)).any():
                df_reddit.loc[k]=new_row
                k+=1
    r.close()
    print("... "+str(round(page_number/n_pages*100,2))+"% done")

df_reddit["website"]="Reddit"
df_reddit.to_csv(path+datetime.datetime.now().strftime("%Y_%m_%d")+"_reddit.csv",index=False)
df_reddit

Starting to scrap The Gear Page, processing the first 5 pages.
... 20.0% done
... 40.0% done
... 60.0% done
... 80.0% done
... 100.0% done


,brand,model,price,vintage,color,wood,post_date,post_title,post_content,post_url,website
0,,,250,,None,,2021.01.19,WTS: Avalanche Run V2 LE Black/White Sparkle,Just an awesome color for this pedal. Mint con...,https://www.reddit.com/r/Gear4Sale/comments/l0...,Reddit
1,,,230,,None,,2021.01.17,WTS Seymour Duncan Jeff Loomis pickup set,https://photos.app.goo.gl/FgspRpYN8GJq3s5w5\n\...,https://www.reddit.com/r/Gear4Sale/comments/ky...,Reddit
2,ibanez,,130,,red,,2021.01.16,"[WTS/WTT] [US-CA] TC Flashback X4, TC Quintess...","r/titlegore, right?\n\nI've made a lot of chan...",https://www.reddit.com/r/Gear4Sale/comments/ky...,Reddit
3,,,350,,None,,2021.01.16,WTS Pro Tools | Duet by Apogee,Pics &amp; timestamp: [https://imgur.com/a/GPY...,https://www.reddit.com/r/Gear4Sale/comments/kx...,Reddit
4,,,600,1978,red,,2021.01.14,WTS: Korg EMX-1SD and ESX-1 Electribes. Teenag...,Looking to unload some of my favorite pieces t...,https://www.reddit.com/r/Gear4Sale/comments/kx...,Reddit
5,aria,,1700,2017,red,alder,2021.01.12,WTS/WTT Kiesel Vader V6 2017 Aquaburst Headles...,ImageURL: [https://imgur.com/a/hFR7shH](https:...,https://www.reddit.com/r/Gear4Sale/comments/kv...,Reddit
6,,,200,2020,None,,2021.01.11,WTS 2020 Mu-Tron Microtron IV $200,"Excellent-near mint condition, velcro on back....",https://www.reddit.com/r/Gear4Sale/comments/ku...,Reddit
7,eko,,150,,None,,2021.01.11,WTS/WTT Eko Jazz Bass,Looking to locally sell or trade a made in Chi...,https://www.reddit.com/r/Gear4Sale/comments/ku...,Reddit
8,,,660,,red,,2021.01.09,WTS: Moog Grandmother Semi-Modular Synth,Picture: [https://imgur.com/a/HoAFmhX](https:/...,https://www.reddit.com/r/Gear4Sale/comments/kt...,Reddit
9,,,425,,None,,2021.01.09,[WTS] Maschine mk3,[US] Maschine Mk3 (used)\n\nUsed Maschine Mk3 ...,https://www.reddit.com/r/Gear4Sale/comments/kt...,Reddit


## The Gear Page

In [338]:
tgp_base_url = "https://www.thegearpage.net"
tgp_url = "https://www.thegearpage.net/board/index.php?forums/guitar-emporium.22/"
df_tgp=pd.DataFrame(columns=["brand","model","price","vintage","color","wood","post_date","post_title","post_content","post_url"])
k=0
n_pages = 5
print("Starting to scrap The Gear Page, processing the first "+str(n_pages)+" pages.")

for page_number in [i+1 for i in range(n_pages)]:
    url = tgp_url + "page-"+str(page_number)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    for div in soup.findAll("div", {"class":"structItem-cell structItem-cell--main"}):
        # check that it is a selling post
        if any(tag in div.text.upper() for tag in ["FS" or "FOS" or "WTS" or "SELL" or "SALE"]):
            title = div.findAll("a", {"class":""})[0].text
            
            # get the post url and from it the content
            post_url = tgp_base_url+div.findAll("a")[1]["href"]
            post_rep = requests.get(post_url)
            post_soup = bs4.BeautifulSoup(post_rep.text, 'html.parser')
            post_date = datetime.datetime.fromtimestamp(float(post_soup.findAll("time",{"class":"u-dt"})[0]["data-time"])).strftime("%Y.%m.%d")
            bbWrappers = post_soup.findAll("div", {"class":"bbWrapper"})
            content = bbWrappers[0].text
            post_rep.close()

            new_row = find_relevant_information(content,title,post_url,post_date)
            if new_row!=[] and not ((df_tgp['post_title'] == title) & (df_tgp['post_content'] == content)).any():
                df_tgp.loc[k]=new_row
                k+=1
    r.close()
    print("... "+str(round(page_number/n_pages*100,2))+"% done")

scrapping_results_file_name = path+datetime.datetime.now().strftime("%Y_%m_%d")+"_theGearPage.csv"
print("Succesfully scraped The Gear Page, saving result to file: "+scrapping_results_file_name)

df_tgp["website"]="TheGearPage"
df_tgp.to_csv(scrapping_results_file_name,index=False)

Starting to scrap The Gear Page, processing the first 5 pages.
... 20.0% done
... 40.0% done
... 60.0% done
... 80.0% done
... 100.0% done
Succesfully scraped The Gear Page, saving result to file: C:/dev/niche/guitar/data/2021_01_19_theGearPage.csv


In [351]:
init_kdb_instance()

In [392]:
insert_scrappedGuitars_into_kdb(df_tgp)

In [354]:
insert_scrappedGuitars_into_kdb(df_reddit)

In [396]:
roll_to_HDB_all_dates()